# MMMI Oversight Script


**TODO**: *Step B.1.* Computation of risky expectations based on historic return information.

**TODO**: *Step C.* Quarterly HARK agent updates.

In [1]:
import HARK.ConsumptionSaving.ConsPortfolioModel as cpm
import logging
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
import sys

sys.path.append('.')

import hark_portfolio_agents as hpa

In [3]:
sys.path.append('../PNL/py')

import util as UTIL
import pnl as pnl

## Initialize the Simulation

### Initialize market information

Some initial values for the simulation.

$RAP_0$ is the starting risky asset price. This is adjust after every trading day.

In [4]:
risky_asset_price = 100

HARK uses numerical values for wealth. This variable converts "HARK money" to dollars.

In [5]:
dollars_per_hark_money_unit = 1000

Expected rate of return and standard deviation of the market price process will be updated over time.

These are the starting values, derived from the S&P 500.

$\bar{r}_0 = 0.000628$

$sr_0 = 0.011988$

In [6]:
sp500_ror = 0.000628
sp500_std = 0.011988

market_rate_of_return = sp500_ror
market_standard_deviation = sp500_std

market_risky_params = {
    'RiskyAvg': market_rate_of_return,
    'RiskyStd': market_standard_deviation,
}


Calendar variables. Will ultimately set to be realistic; for now set low for testing.

In [7]:
days_per_quarter = 6
quarters_per_simulation = 4

An attention rate: the chance on any day that an agent will pay attention to the market and trade.

Later, this can be a more realistic function of agent circumstances.

In [8]:
attention_rate = 1 / days_per_quarter

### Initialize agents to start

This is to set up the agent parameters and quantities used in the simulation.

`CRRA` refers to the `\rho` parameter in the constant relative risk aversion utility function used by the agents. Utility is over units of resources consumed.

`aNrmInitStd` is normalized initial market assets, standard deviation. The agents' starting wealth is drawn from a lognormal distribution with this standard deviation.

`pLvlInitMean` and `pLvlInitStd` are the mean and standard deviation of the initial level of permanent income of the agents.

In [9]:
## List of tuples: 
##  ( parameters, (i.e. coefficient of relative risk aversion CRRA)
##    number of agents represented,
##    ( initial risky percent, $$$ in risky asset, $$$ in riskless asset)
agent_classes = [
    {
        'CRRA': .9,
        'AgentCount' : 10,
        'aNrmInitMean' : 10
    }] * 10 + \
    [{
        'CRRA': .95, 
        'AgentCount' : 10,
        'aNrmInitMean': 10
    }] * 10 + \
    [{
        'CRRA': .99, 
        'AgentCount' : 10,
        'aNrmInitMean' : 10
    }] * 10


### parameters shared by all agents
agent_parameters = {
    'aNrmInitStd' : 0.0,
    'pLvlInitMean' : 1.0, # initial distribution of permanent income
    'pLvlInitStd' : 0.0,
    'RiskyAvg': market_rate_of_return,
    'RiskyStd': market_standard_deviation,    
}

In [10]:
len(agent_classes)

30

Initialize the agents.

In [11]:
agents = hpa.create_agents(agent_classes, agent_parameters)

## Get agent risky allocations

Use this stock price to determine the initial number of shares of the risky asset held by each
agent:
 - $RAS_{i0} = (HEC_{i0}.RAW)/ RAP_0$
 
 where RAW is Risky Asset Wealth, amount of resources allocated to the risky asset.
 
 RAS is Risky Asset Shares, number of shares the agent has.

In [12]:
def compute_share_demand(agent):
    agent.solve()
    market_resources = agent.state_now['mNrm']
    permanent_income = agent.state_now['pLvl']

    # ShareFunc takes normalized market resources as argument
    risky_share = agent.solution[0].ShareFuncAdj(
        market_resources
    )
    
    # denormalize the risky share. See https://github.com/econ-ark/HARK/issues/986
    risky_asset_wealth = risky_share \
                        * market_resources \
                        * permanent_income \
                        * dollars_per_hark_money_unit 
    
    shares = risky_asset_wealth / risky_asset_price
    
    return shares

In [13]:
for agent in agents:
    agent.shares = compute_share_demand(agent)

## Prepare the main loop

Functions to automate aspects of the main loop.
(Will move this to module code once finalized).

### Activating agents

"At the beginning of each trading day $t + 1$, select a random group of HARK agents with replacement."

**TODO**: HARK AgentType classes represent multiple agents (as many as `AgentCount`). It is difficult to have the agents within the same AgentType act 'separately', though these agents will have _ex ante_ and _ex post_ heterogeneity of some variables. For the purpose of MMMI, we may need to scaffold the agent model to make it easier to have multiple independently acting agents.

In [14]:
import random

def activate(agents, attention_rate):
    for agent in agents:
        agent.attentive = random.random() < attention_rate

In [15]:
activate(agents, attention_rate)

"**First**, determine the expected return and standard deviation of the risky asset to be used for decision period $t + 1$."

In [16]:
def calculate_risky_expectations():
    # TODO: This should be an adaptive function of market prices
    # as described in John's Experiment design.
    # This is a stopgap for now.
    
    return {
        'RiskyAvg': market_rate_of_return,
        'RiskyStd': market_standard_deviation
    }
    

"**Second**, using [these expectations] calculate the total number of shares of the risky asset that the selected agents want to buy and sell."

In [17]:
def compute_total_buy_sell(agents):
    """
    TODO: Should this be done without making so many changes to the underlying
    agent? Could be done using copies.
    """
    risky_expectations = calculate_risky_expectations()
    
    buy_shares_total = 0
    sell_shares_total = 0
    
    for agent in agents:
        if agent.attentive:
            # Note: this mutates the underlying agent
            agent.assign_parameters(**risky_expectations)
            
            d_shares = compute_share_demand(agent)
            
            
            delta_shares = d_shares - agent.shares
            
            # NOTE: This mutates the agent
            agent.shares = d_shares
            
            buy_shares_total += delta_shares[delta_shares > 0].sum()
            sell_shares_total += -delta_shares[delta_shares < 0].sum()
            
    return (round(buy_shares_total), round(sell_shares_total))

In [18]:
compute_total_buy_sell(agents)

(0, 0)

"**Third**, using [total buy and sell shares] as input to the Buy Broker and Sell Broker, run a day of NetLogo trading and determine the end of day price of the risky asset."

In [28]:
import os

def run_market(buy_sell, seed = None):
    output_path = f"../PNL/out/logs/LMtransactions_{seed}.csv"
    if os.path.exists(output_path):
        print(f"Transaction output for {seed} already exists. Will use cache.")
        return
    
    
    config = UTIL.read_config(
        config_file = "../PNL/macroliquidity.ini",
        config_local_file = "../PNL/macroliquidity_local.ini"
    )
    pnl.run_NLsims(
        config,
        broker_buy_limit = buy_sell[0],
        broker_sell_limit = buy_sell[1],
        SEED = seed
    )

In [30]:
run_market((0,0),seed=1)

Transaction output for 1 already exists. Will use cache.


"**Fourth**, use the final price to calculate the amount of wealth each (all) HARK agent has allocated to the risky asset."

In [31]:
def get_transactions(seed=0):
    # use run_market() first to create logs
    transactions = pd.read_csv(
        f"../PNL/out/logs/LMtransactions_{seed}.csv",
        delimiter='\t'
    )
    return transactions

def get_last_simulation_price(seed=0):
    """
    Dividing the output of this by 4
    """
    
    transactions = get_transactions(seed=seed)
    prices = transactions['TrdPrice']
    
    return prices[prices.index.values[-1]]

In [32]:
netlogo_ror = -0.00052125
netlogo_std =  0.0068
simulation_price_scale = 0.25

def daily_rate_of_return(seed = 0):
    
    last_sim_price = get_last_simulation_price(seed=seed)
    
    ror = (last_sim_price * simulation_price_scale - 100) / 100
    
    # adjust to calibrated NetLogo to S&P500
    ror = sp500_std * (ror - netlogo_ror) / netlogo_std + sp500_ror
    
    return ror

In [33]:
def update_agent_wealth(old_share_price, ror):

    new_share_price = old_share_price * (1 + ror)

    for agent in agents:
        market_resources = agent.state_now['mNrm'] * agent.state_now['pLvl']
    
        old_raw = agent.shares * old_share_price
        new_raw = agent.shares * new_share_price
    
        delta_mNrm = (new_raw - old_raw) / \
            (dollars_per_hark_money_unit * agent.state_now['pLvl'])
    
        agent.state_now['mNrm'] = agent.state_now['mNrm'] + delta_mNrm

"Repeat each trading day until the end of the quarter. Then update the economic conditions for each HARK agent (e.g., permanent income level), and repeat [daily loop]."

## Run the main loop



In [34]:
prices = []
ror_list = [sp500_ror]
std_list = [sp500_std]


# Main loop
for quarter in range(quarters_per_simulation):
    for day in range(days_per_quarter):
        print(f"Q-{quarter}:D-{day}")
        
        seed = 0
        prices.append(risky_asset_price)
        
        activate(agents, attention_rate)
        buy_sell = compute_total_buy_sell(agents)
        run_market(buy_sell, seed = seed)
        
        ror =  daily_rate_of_return(seed = seed)
        print("ror: " + str(ror))
        ror_list.append(ror)
        
        update_agent_wealth(risky_asset_price, ror)
        
        risky_asset_price = risky_asset_price * (1 + ror)

Q-0:D-0
Transaction output for 0 already exists. Will use cache.
ror: 0.005954286029411765
Q-0:D-1
Transaction output for 0 already exists. Will use cache.
ror: 0.005954286029411765
Q-0:D-2
Transaction output for 0 already exists. Will use cache.
ror: 0.005954286029411765
Q-0:D-3
Transaction output for 0 already exists. Will use cache.
ror: 0.005954286029411765
Q-0:D-4
Transaction output for 0 already exists. Will use cache.
ror: 0.005954286029411765
Q-0:D-5
Transaction output for 0 already exists. Will use cache.
ror: 0.005954286029411765
Q-1:D-0
Transaction output for 0 already exists. Will use cache.
ror: 0.005954286029411765
Q-1:D-1
Transaction output for 0 already exists. Will use cache.
ror: 0.005954286029411765
Q-1:D-2
Transaction output for 0 already exists. Will use cache.
ror: 0.005954286029411765
Q-1:D-3
Transaction output for 0 already exists. Will use cache.
ror: 0.005954286029411765
Q-1:D-4
Transaction output for 0 already exists. Will use cache.
ror: 0.005954286029411765

In [35]:
prices

[103.62617599157338,
 104.24319588356136,
 104.86388968847207,
 105.48827928183391,
 106.11638666942842,
 106.74823398806585,
 107.38384350636537,
 108.02323762553985,
 108.66643888018542,
 109.31346993907563,
 109.96435360596038,
 110.61911282036965,
 111.2777706584219,
 111.94035033363743,
 112.60687519775645,
 113.27736874156216,
 113.95185459570857,
 114.63035653155335,
 115.31289846199566,
 115.9995044423189,
 116.69019867103847,
 117.38500549075472,
 118.08394938901074,
 118.78705499915549]

In [36]:
ror_list

[0.000628,
 0.005954286029411765,
 0.005954286029411765,
 0.005954286029411765,
 0.005954286029411765,
 0.005954286029411765,
 0.005954286029411765,
 0.005954286029411765,
 0.005954286029411765,
 0.005954286029411765,
 0.005954286029411765,
 0.005954286029411765,
 0.005954286029411765,
 0.005954286029411765,
 0.005954286029411765,
 0.005954286029411765,
 0.005954286029411765,
 0.005954286029411765,
 0.005954286029411765,
 0.005954286029411765,
 0.005954286029411765,
 0.005954286029411765,
 0.005954286029411765,
 0.005954286029411765,
 0.005954286029411765]

# Old Code below : to update 

In [ ]:
# STEP 2. Create starting demand for the market.
#         Burn in the new prices.


buy_sell = hpa.aggregate_buy_and_sell(
    hpa.no_demand(agents), # zero allocated to risky asset
    init_demands
)


### Agents estimate return rate and volatility from historical data

The agents look at the most recent historical price data to form their expectations of the future.

This is a very simplified version of what could later be better specified in line with _adaptive expectations_:

https://en.wikipedia.org/wiki/Adaptive_expectations

### Demand function for the intermittent demand problem

In the intermittent demand problem, consumers take their current state and determine risky porfolio share of their assets to invest.

The agent responds to the price history by:
 - updating their current assets
 - updating their expectations of the risky asset
 - returning their risky share allocation.

Other aspects of the problem are fixed for now.

### Putting it all together

Each agent knows their old risky allocation, old risk free allocation, and the prices of the last quarter.

They computer their new market assets and then their new demand.

In [ ]:
# STEP 3: Create the starting asset price. This will be used
#         to translate $$$ values to asset quantities
#         and adjust each period.

asset_price = 100

print(f"Initial Buy/Sell: {buy_sell}")

## TODO: ABM Group reimplement the run_market method
transactions = get_transactions()
print(hpa.risky_expectations(transactions))

In [ ]:
plt.plot(transactions['TrdPrice'])
plt.title("Transaction price over time")
plt.show()


old_demands = init_demands

In [ ]:
# STEP 4. Run simulation for N rebalances

N = 10

for i in range(N):
    
    print(f"Asset price: {asset_price}")

    # simulate one period on the macro side
    hpa.simulate(agents, 1)
    
    hpa.update_agents(agents, transactions)
    print(hpa.risky_expectations(transactions))
    new_demands = hpa.demands(agents, transactions)
    
    buy_sell = hpa.aggregate_buy_and_sell(
        old_demands,
        new_demands
    )
    print(f"Round {i} Buy/Sell: {buy_sell}")

    #run_market(buy_sell, asset_price)
    transactions = get_transactions()
    print(hpa.risky_expectations(transactions))
    
    plt.plot(transactions['TrdPrice'])
    plt.show()
    
    old_demands = new_demands
    
    risky_return = hpa.risky_actual_return(transactions)
    asset_price = asset_price * risky_return[0]

In [ ]:
agents[0].history['Risky'][:,0]

In [ ]:
plt.plot(transactions[transactions['OrderQuantity'] > 0]['OrderPrice'])

In [ ]:
transactions['OrderQuantity'] > 0

In [ ]:
transactions

In [ ]:
buys = transactions[(transactions['OrderBA'] == "Buy") & (transactions['OrderQuantity'] > 0)]
sells = transactions[(transactions['OrderBA'] == "Sell") & (transactions['OrderQuantity'] > 0)]


plt.plot(buys['Tick'], buys['OrderPrice'])
plt.plot(sells['Tick'], sells['OrderPrice'])


In [ ]:
sells

In [ ]:
(transactions['OrderPrice'] * transactions['OrderQuantity']).sum() / transactions['OrderQuantity'].sum()

In [ ]:
hpa.risky_expectations(transactions)

In [ ]:
transactions[(transactions['OrderBA'] == "Buy") & (transactions['OrderQuantity'] > 0)]

In [ ]:
agents[0].solution[0].ShareFuncAdj(0)

In [ ]:
agents[0].state_now['mNrm']

In [ ]:
agents[0].state_now['pLvl']

In [ ]:
new_assets(agent, hpa.risky_share, prices)

In [ ]:
transactions = pd.read_csv("../PNL/out/logs/LMalltransactions_90_5.csv", delimiter='\t')

In [ ]:
transactions

In [ ]:
plt.plot(transactions['OrderPrice'])

In [ ]:
transactions['TraderWhoType'].unique()

In [ ]:
sales = transactions[transactions['TraderWhoType'] == 'LiqSellBkr']
avg_sale_price = sales['OrderQuantity'] * sales['OrderPrice'] / sales['OrderPrice'].sum()

In [ ]:
transactions[transactions['TraderWhoType'] == 'LiqBuyBkr']

In [ ]:
buys = transactions[transactions['TraderWhoType'] == 'LiqBuyBkr]
avg_buy_price = buys['OrderQuantity'] * buys['OrderPrice'] / buys['OrderPrice'].sum()

In [ ]:
agents[0].history['mNrm'][0][-1] - agents[0].history['mNrm'][0][0]

In [ ]:
agents[2].controls['Share']

In [ ]:
transactions

In [ ]:
transactions[transactions['OrderQuantity'] == 0].groupby('Tick').size()